# [모의 캐글 - 게임] 비매너 댓글 식별 

- 자연어 multi label classification 과제
- 작성자 : MNC Sukyung Kim (skkim@mnc.ai)

참고 논문 : 
- [BERT: Pre-training of Deep Bidirectional Transformers for
Language Understanding](https://arxiv.org/pdf/1810.04805.pdf)
- [Attention Is All You Need](https://arxiv.org/pdf/1706.03762.pdf)

# 1. 환경 설정 및 라이브러리 불러오기

In [1]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 29.8 MB/s eta 0:00:0000:01:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.8/121.8 KB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 41.8 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.6/764.6 KB 15.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 895.2/895.2 KB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 KB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 KB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 66.7 MB/s eta 0:00:00


In [1]:
import pandas as pd
import os
import json
import numpy as np
import shutil

from sklearn.metrics import f1_score
from datetime import datetime, timezone, timedelta
import random
from tqdm import tqdm


from attrdict import AttrDict
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils import *
from torch.optim import Adam, AdamW

from transformers import logging, get_linear_schedule_with_warmup


from transformers import ( 
    BertConfig,
    ElectraConfig,
    ElectraConfig
)

from transformers import (
    BertTokenizer,  
    AutoTokenizer,
    ElectraTokenizer,
)

from transformers import (
    BertModel,
    AutoModel, 
    ElectraForSequenceClassification,
    BertForSequenceClassification
)
from hanspell import spell_checker
from pykospacing import Spacing
from soynlp.normalizer import *

2022-02-23 16:04:24.968852: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-02-23 16:04:25.941033: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-02-23 16:04:25.969893: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:86:00.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2022-02-23 16:04:25.969946: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-02-23 16:04:25.972775: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2022-02-23 16:04:25.972842: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
202

In [2]:
# 사용할 GPU 지정
print("number of GPUs: ", torch.cuda.device_count())
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
use_cuda = torch.cuda.is_available()
print("Does GPU exist? : ", use_cuda)
DEVICE = torch.device("cuda" if use_cuda else "cpu")

number of GPUs:  1
Does GPU exist? :  True


In [3]:
# True 일 때 코드를 실행하면 example 등을 보여줌
DEBUG = True

In [4]:
# config 파일 불러오기
config_path = os.path.join('config.json')

def set_config(config_path):
    if os.path.lexists(config_path):
        with open(config_path) as f:
            args = AttrDict(json.load(f))
            print("config file loaded.")
            print(args.pretrained_model)
    else:
        assert False, 'config json file cannot be found.. please check the path again.'
    
    return args
    

# 코드 중간중간에 끼워넣어 리셋 가능
args = set_config(config_path)

# 결과 저장 폴더 미리 생성
os.makedirs(args.result_dir, exist_ok=True)
os.makedirs(args.config_dir, exist_ok=True)

config file loaded.
beomi/kcbert-base


# 2. EDA 및 데이터 전처리

In [5]:
# data 경로 설정  
train_path = os.path.join(args.data_dir,'/USER/team_project3/data/train.csv')

print("train 데이터 경로가 올바른가요? : ", os.path.lexists(train_path))


train 데이터 경로가 올바른가요? :  True


### 2-1. Train 데이터 확인

In [6]:
train_df = pd.read_csv(train_path, encoding = 'UTF-8-SIG')

train_df.head()

,title,comment,bias,hate
0,"""'미스터 션샤인' 변요한, 김태리와 같은 양복 입고 학당 방문! 이유는?""",김태리 정말 연기잘해 진짜,none,none
1,"""[SC현장]""""극사실주의 현실♥""""…'가장 보통의 연애' 김래원X공효진, 16년만...",공효진 발연기나이질생각이읍던데 왜계속주연일까,none,hate
2,"""손연재, 리듬체조 학원 선생님 """"하고 싶은 일 해서 행복하다""""""",누구처럼 돈만 밝히는 저급인생은 살아가지마시길~~ 행복은 머니순이 아니니깐 작은거에...,others,hate
3,"""'섹션TV' 김해숙 """"'허스토리' 촬영 후 우울증 얻었다""""""",일본 축구 져라,none,none
4,"""[단독] 임현주 아나운서 “‘노브라 챌린지’ 방송 덕에 낸 용기, 자연스런 논의의...",난 절대로 임현주 욕하는인간이랑은 안논다 @.@,none,none


In [7]:
len(train_df)

8367

In [8]:
print("bias classes: ", train_df.bias.unique())
print("hate classes: ", train_df.hate.unique())

bias classes:  ['none' 'others' 'gender']
hate classes:  ['none' 'hate']


In [9]:
pd.crosstab(train_df.bias, train_df.hate, margins=True)

hate,hate,none,All
bias,,,
gender,1216,83,1299
none,2068,3422,5490
others,1437,141,1578
All,4721,3646,8367


### 2-2. Test 데이터 확인

In [10]:
test_path = os.path.join(args.data_dir,'/USER/team_project3/data/test.csv')
print("test 데이터 경로가 올바른가요? : ", os.path.lexists(test_path))

test 데이터 경로가 올바른가요? :  True


In [11]:
test_df = pd.read_csv(test_path)
test_df.head()

,ID,title,comment
0,0,"류현경♥︎박성훈, 공개연애 4년차 애정전선 이상無..""의지 많이 된다""[종합]",둘다 넘 좋다~행복하세요
1,1,"""현금 유도+1인 1라면?""…'골목식당' 백종원, 초심 잃은 도시락집에 '경악' [종합]",근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데
2,2,"입대 D-11' 서은광의 슬픈 멜로디..비투비, 눈물의 첫 체조경기장[콘서트 종합]",누군데 얘네?
3,3,"아이콘택트' 리쌍 길, 3년 전 결혼설 부인한 이유 공개…""결혼,출산 숨겼다""","쑈 하지마라 짜식아!음주 1번은 실수, 2번은 고의, 3번은 인간쓰레기다.슬금슬금 ..."
4,4,"구하라, 안검하수 반박 해프닝...""당당하다""vs""그렇게까지"" 설전 [종합]",안검하수 가지고 있는 분께 희망을 주고 싶은건가요? 수술하면 이렇게 자연스러워진다고...


In [12]:
len(test_df)

511

### 2-3. 데이터 전처리 (Label Encoding)
bias, hate 라벨들의 class를 정수로 변경하여 라벨 인코딩을 하기 위한 딕셔너리입니다.

- bias, hate 컬럼을 합쳐서 하나의 라벨로 만들기 

In [13]:
# 두 라벨의 가능한 모든 조합 만들기
combinations = np.array(np.meshgrid(train_df.bias.unique(), train_df.hate.unique())).T.reshape(-1,2)

if DEBUG==True:
    print(combinations)

[['none' 'none']
 ['none' 'hate']
 ['others' 'none']
 ['others' 'hate']
 ['gender' 'none']
 ['gender' 'hate']]


In [14]:
# bias, hate 컬럼을 합친 것
bias_hate = list(np.array([train_df['bias'].values, train_df['hate'].values]).T.reshape(-1,2))

if DEBUG==True:
    print(bias_hate[:5])


[array(['none', 'none'], dtype=object), array(['none', 'hate'], dtype=object), array(['others', 'hate'], dtype=object), array(['none', 'none'], dtype=object), array(['none', 'none'], dtype=object)]


In [15]:
labels = []
for i, arr in enumerate(bias_hate):
    for idx, elem in enumerate(combinations):
        if np.array_equal(elem, arr):
            labels.append(idx)

train_df['label'] = labels
train_df.head()

,title,comment,bias,hate,label
0,"""'미스터 션샤인' 변요한, 김태리와 같은 양복 입고 학당 방문! 이유는?""",김태리 정말 연기잘해 진짜,none,none,0
1,"""[SC현장]""""극사실주의 현실♥""""…'가장 보통의 연애' 김래원X공효진, 16년만...",공효진 발연기나이질생각이읍던데 왜계속주연일까,none,hate,1
2,"""손연재, 리듬체조 학원 선생님 """"하고 싶은 일 해서 행복하다""""""",누구처럼 돈만 밝히는 저급인생은 살아가지마시길~~ 행복은 머니순이 아니니깐 작은거에...,others,hate,3
3,"""'섹션TV' 김해숙 """"'허스토리' 촬영 후 우울증 얻었다""""""",일본 축구 져라,none,none,0
4,"""[단독] 임현주 아나운서 “‘노브라 챌린지’ 방송 덕에 낸 용기, 자연스런 논의의...",난 절대로 임현주 욕하는인간이랑은 안논다 @.@,none,none,0


In [16]:
train_df["title"] = train_df["title"].str.replace(pat=r'[^\w]', repl=r'', regex=True)

In [17]:
train_df["comment"] = train_df["comment"].str.replace(pat=r'[^\w]', repl=r'', regex=True)

In [18]:
train_df

,title,comment,bias,hate,label
0,미스터션샤인변요한김태리와같은양복입고학당방문이유는,김태리정말연기잘해진짜,none,none,0
1,SC현장극사실주의현실가장보통의연애김래원X공효진16년만의랑데부종합,공효진발연기나이질생각이읍던데왜계속주연일까,none,hate,1
2,손연재리듬체조학원선생님하고싶은일해서행복하다,누구처럼돈만밝히는저급인생은살아가지마시길행복은머니순이아니니깐작은거에감사하고항상좋은일만...,others,hate,3
3,섹션TV김해숙허스토리촬영후우울증얻었다,일본축구져라,none,none,0
4,단독임현주아나운서노브라챌린지방송덕에낸용기자연스런논의의창됐으면인터뷰,난절대로임현주욕하는인간이랑은안논다,none,none,0
...,...,...,...,...,...
8362,배우이필립SNS스타연인에게초호화프러포즈눈길,아니근데튜닝한사람은프러포즈받지도결혼도못함ㅋㅋㅋ지들은돈없어서못하는것들이ㅋㅋㅋㅋ아이고배...,others,hate,3
8363,SC이슈마약백스텝김새롬탓실형피한이찬오이미지는치명상종합,그러니깐여자를잘만나야되징글징글한것들만나면인생끝가지돌아가게되는듯근데왜다들김새롬편을들어...,gender,hate,5
8364,POP이슈그들만의세상홍상수김민희새해데이트에반응싸늘,참으로아름다운커플입니다늘행복하시고새해에도늘꽃길만걸으시길축원합니다,none,none,0
8365,종합시크릿마더김소연누가죽였나송윤아와갈등,재미가없어요,none,none,0


In [19]:
spacing = Spacing()

In [49]:
title_list = []
for i, title in enumerate(train_df["title"]):
        title_list.append(spacing(title))
        if i % 500 == 0:
            print(f"{i}번쨰 진행중...")

0번쨰 진행중...
500번쨰 진행중...
1000번쨰 진행중...
1500번쨰 진행중...
2000번쨰 진행중...
2500번쨰 진행중...
3000번쨰 진행중...
3500번쨰 진행중...
4000번쨰 진행중...
4500번쨰 진행중...
5000번쨰 진행중...
5500번쨰 진행중...
6000번쨰 진행중...
6500번쨰 진행중...
7000번쨰 진행중...
7500번쨰 진행중...
8000번쨰 진행중...


In [51]:
train_df["title"] = title_list

In [53]:
comment_list = []
for i, comment in enumerate(train_df["comment"]):
        comment_list.append(spacing(comment))
        if i % 500 == 0:
            print(f"{i}번쨰 진행중...")

0번쨰 진행중...
500번쨰 진행중...
1000번쨰 진행중...
1500번쨰 진행중...
2000번쨰 진행중...
2500번쨰 진행중...
3000번쨰 진행중...
3500번쨰 진행중...
4000번쨰 진행중...
4500번쨰 진행중...
5000번쨰 진행중...
5500번쨰 진행중...
6000번쨰 진행중...
6500번쨰 진행중...
7000번쨰 진행중...
7500번쨰 진행중...
8000번쨰 진행중...


In [55]:
train_df["comment"] = comment_list

In [57]:
train_df

,title,comment,bias,hate,label
0,미스터션 샤인변 요한 김태리와 같은 양복 입고 학당 방문 이유는,김태리 정말 연기 잘해진짜,none,none,0
1,SC 현장 극사실주의 현실 가장 보통의 연애 김래원 X공효진 16년 만의 랑데부종합,공효진발 연기나 이질 생각이 읍던데 왜 계속 주연일까,none,hate,1
2,손연재 리듬체조 학원 선생님하고 싶은 일해서 행복하다,누구처럼 돈만 밝히는 저급 인생은 살아가지 마시길 행복은 머니 순이 아니니깐 작은 ...,others,hate,3
3,섹션 TV 김해숙 허스토리 촬영 후 우울증 얻었다,일본 축구 져라,none,none,0
4,단독 임현주 아나운서 노브라 챌린지 방송 덕에 낸 용기 자연스런 논의의 창됐으면 인터뷰,난 절대로 임현주 욕하는 인간이랑은 안 논다,none,none,0
...,...,...,...,...,...
8362,배우 이필립 SNS 스타 연인에게 초호화 프러포즈 눈길,아니 근데 튜닝한 사람은 프러포즈 받지도 결혼도 못함 ㅋㅋㅋ지들은 돈 없어서 못하는...,others,hate,3
8363,SC 이슈 마약백스텝 김새롬 탓 실형 피한 이찬오 이미지는 치명상종합,그러니깐 여자를 잘 만나야 되징글징글한 것들 만나면 인생 끝가지 돌아가게 되는 듯 ...,gender,hate,5
8364,POP 이슈 그들만의 세상 홍상수 김민희 새해 데이트에 반응 싸늘,참으로 아름다운 커플입니다 늘 행복하시고 새해에도 늘 꽃길만 걸으시길 축원합니다,none,none,0
8365,종합시크릿마더 김소연 누가 죽였나 송윤아와 갈등,재미가 없어요,none,none,0


In [80]:
spell_comment_list = []
for i, title in enumerate(train_df["title"]):
        spelled_title = spell_checker.check(title)
        hanspell_title = spelled_title.checked
        spell_title_list.append(hanspell_title)
        if i % 500 == 0:
            print(f"{i}번쨰 진행중...")

0번쨰 진행중...
500번쨰 진행중...
1000번쨰 진행중...
1500번쨰 진행중...
2000번쨰 진행중...
2500번쨰 진행중...
3000번쨰 진행중...
3500번쨰 진행중...
4000번쨰 진행중...
4500번쨰 진행중...
5000번쨰 진행중...
5500번쨰 진행중...
6000번쨰 진행중...
6500번쨰 진행중...
7000번쨰 진행중...
7500번쨰 진행중...
8000번쨰 진행중...


In [82]:
train_df["title"] = spell_title_list

In [85]:
spell_comment_list = []
for i, comment in enumerate(train_df["comment"]):
        spelled_comment = spell_checker.check(comment)
        hanspell_comment = spelled_comment.checked
        spell_comment_list.append(hanspell_comment)
        if i % 500 == 0:
            print(f"{i}번쨰 진행중...")

0번쨰 진행중...
500번쨰 진행중...
1000번쨰 진행중...
1500번쨰 진행중...
2000번쨰 진행중...
2500번쨰 진행중...
3000번쨰 진행중...
3500번쨰 진행중...
4000번쨰 진행중...
4500번쨰 진행중...
5000번쨰 진행중...
5500번쨰 진행중...
6000번쨰 진행중...
6500번쨰 진행중...
7000번쨰 진행중...
7500번쨰 진행중...
8000번쨰 진행중...


In [87]:
train_df["comment"] = spell_comment_list

In [88]:
train_df

,title,comment,bias,hate,label
0,미스터 션 샤인 변 요한 김태리와 같은 양복 입고 학당 방문 이유는,김태리 정말 연기 잘해 진짜,none,none,0
1,SC 현장 극사실주의 현실 가장 보통의 연애 김래원 X 공효진 16년 만의 랑데부 종합,공효진 발 연기나 이질 생각이 없던데 왜 계속 주연일까,none,hate,1
2,손연재 리듬체조 학원 선생님 하고 싶은 일 해서 행복하다,누구처럼 돈만 밝히는 저급 인생은 살아가지 마시길 행복은 머니 순이 아니니깐 작은 ...,others,hate,3
3,섹션 TV 김해숙 허스토리 촬영 후 우울증 얻었다,일본 축구 져라,none,none,0
4,단독 임현주 아나운서 노브라 챌린지 방송 덕에 낸 용기 자연스러운 논의의 창 됐으면...,난 절대로 임현주 욕하는 인간이랑은 안 논다,none,none,0
...,...,...,...,...,...
8362,배우 이필립 SNS 스타 연인에게 초호화 프러포즈 눈길,아니 근데 튜닝한 사람은 프러포즈 받지도 결혼도 못함 ᄏᄏᄏ 자기들은 돈 없어서 못...,others,hate,3
8363,SC 이슈 마약 백스텝 김새롬 탓 실형 피한 이찬오 이미지는 치명상 종합,그러니깐 여자를 잘 만나야 돼 징글징글한 것들 만나면 인생 끝가지 돌아가게 되는 듯...,gender,hate,5
8364,POP 이슈 그들만의 세상 홍상수 김민희 새해 데이트에 반응 싸늘,참으로 아름다운 커플입니다 늘 행복하시고 새해에도 늘 꽃길만 걸으시길 축원합니다,none,none,0
8365,종합 시크릿 마더 김소연 누가 죽였나 송윤아와 갈등,재미가 없어요,none,none,0


In [96]:
repeat_comment_list = []
for i, comment in enumerate(train_df["comment"]):
        repeat_comment = repeat_normalize(comment, num_repeats=2)
        repeat_comment_list.append(repeat_comment)
        if i % 500 == 0:
            print(f"{i}번쨰 진행중...")

0번쨰 진행중...
500번쨰 진행중...
1000번쨰 진행중...
1500번쨰 진행중...
2000번쨰 진행중...
2500번쨰 진행중...
3000번쨰 진행중...
3500번쨰 진행중...
4000번쨰 진행중...
4500번쨰 진행중...
5000번쨰 진행중...
5500번쨰 진행중...
6000번쨰 진행중...
6500번쨰 진행중...
7000번쨰 진행중...
7500번쨰 진행중...
8000번쨰 진행중...


In [191]:
train_df["comment"][8362] #soynlp

'아니 근데 튜닝한 사람은 프러포즈 받지도 결혼도 못함 ᄏᄏᄏ 자기들은 돈 없어서 못하는 것들이 ㅋㅋㅋㅋ 아이고 배 아파죽지ᄏ'

In [193]:
repeat_normalize(train_df["comment"][8362], num_repeats=1)

'아니 근데 튜닝한 사람은 프러포즈 받지도 결혼도 못함 ᄏᄏᄏ 자기들은 돈 없어서 못하는 것들이 ㅋ 아이고 배 아파죽지ᄏ'

In [201]:
repeat_normalize("와하하하하하하하하하하", num_repeats=2)

'와하하'

In [116]:
train_df.to_csv('train(전처리).csv', index=False)

## 3. Dataset 로드

### 3-0. Pre-trained tokenizer 탐색

In [161]:
from konlpy.tag import Okt

In [190]:
# okt = Okt()
# example = "고기를 아이구 아무렇게나 구우려고 하면 안 돼. 고기라고 다 같은 게 아니거든. 예컨대 삼겹살을 구울 때는 중요한 게 있지."
# stop_words = "이 있 하 것 들 그 되 수 이 보 않 없 나 사람 주 아니 등 같 우리 때 년 가 한 지대하 오 말 일 그렇 위하 때문 그것 두 말하 알 그러나 받 못하 일 그런 또 문제 더 사회 많 그리고 "

# stop_words = set(example.split(' '))
# word_tokens = okt.morphs(example)

# result = [word for word in word_tokens if not word in stop_words]

# print('불용어 제거 전 :',word_tokens) 
# print('불용어 제거 후 :',result)

In [169]:
word_tokens = okt.pos(train_df["comment"][0])

In [180]:
stop_words = ""
for comment in train_df["comment"]:
    word_tokens = okt.pos(comment)
    
    for token in word_tokens:
        if token[1] == "Josa":
            stop_words += (" " + token[0])

In [184]:
a = set(stop_words)

In [187]:
stop_words = ""
for word in a:
    stop_words += (" " + word)

In [188]:
print(stop_words)

 겐 고 만 게 럽 인 데 은 대 처 밖 와 의 기 든 에 써 하 깐 단 냐 다 군 예 터 테 래 롭 러 큼 가 즉 구 같 며 차 럼 커 네 엘   스 선 녕 로 도 니 운 치 나 과 론 는 아 곤 저 랑 서 냥 면 께 요 던 여 진 어 을 이 죠 보 지 마 난 를 으 엔 슨 말 들 까 한 야 라 란 조 턴 부


In [ ]:
stop_words = set(example.split(' '))
word_tokens = okt.morphs(example)

result = [word for word in word_tokens if not word in stop_words]

In [117]:
# config.json 에서 지정 이름별로 가져올 라이브러리 지정

TOKENIZER_CLASSES = {
    "BertTokenizer": BertTokenizer
}


- Tokenizer 사용 예시

In [118]:
TOKENIZER = TOKENIZER_CLASSES[args.tokenizer_class].from_pretrained(args.pretrained_model)
if DEBUG==True:
    print(TOKENIZER)


PreTrainedTokenizer(name_or_path='beomi/kcbert-base', vocab_size=30000, model_max_len=300, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


In [138]:
train_df['title'][1]

'SC 현장 극사실주의 현실 가장 보통의 연애 김래원 X 공효진 16년 만의 랑데부 종합'

In [139]:
if DEBUG == True:
    example = train_df['title'][1]
    print(TOKENIZER(example))

{'input_ids': [2, 52, 4881, 13623, 392, 10432, 8055, 8454, 8547, 10766, 4042, 13853, 420, 4369, 4165, 57, 314, 4894, 4153, 11678, 4482, 1296, 4042, 1125, 4092, 4131, 18606, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [140]:
if DEBUG==True:
    print(TOKENIZER.encode(example),"\n")
    
    # 토큰으로 나누기
    print(TOKENIZER.tokenize(example),"\n")
    
    # 토큰 id로 매핑하기
    print(TOKENIZER.convert_tokens_to_ids(TOKENIZER.tokenize(example)))


[2, 52, 4881, 13623, 392, 10432, 8055, 8454, 8547, 10766, 4042, 13853, 420, 4369, 4165, 57, 314, 4894, 4153, 11678, 4482, 1296, 4042, 1125, 4092, 4131, 18606, 3] 

['S', '##C', '현장', '극', '##사실', '##주의', '현실', '가장', '보통', '##의', '연애', '김', '##래', '##원', 'X', '공', '##효', '##진', '16', '##년', '만', '##의', '랑', '##데', '##부', '종합'] 

[52, 4881, 13623, 392, 10432, 8055, 8454, 8547, 10766, 4042, 13853, 420, 4369, 4165, 57, 314, 4894, 4153, 11678, 4482, 1296, 4042, 1125, 4092, 4131, 18606]


### 3-1. Dataset 만드는 함수 정의

In [ ]:
class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len, mode = 'train'):

        self.data = df
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.mode = mode
        
        if self.mode!='test':
            try: 
                self.labels = df['label'].tolist()
            except:
                assert False, 'CustomDataset Error : \'label\' column does not exist in the dataframe'
     
    def __len__(self):
        return len(self.data)
                

    def __getitem__(self, idx):
        """
        전체 데이터에서 특정 인덱스 (idx)에 해당하는 기사제목과 댓글 내용을 
        토크나이즈한 data('input_ids', 'attention_mask','token_type_ids')의 딕셔너리 형태로 불러옴
        """
        title = self.data.title.iloc[idx]
        comment = self.data.comment.iloc[idx]
        
        tokenized_text = self.tokenizer(title, comment,
                             padding= 'max_length',
                             max_length=self.max_len,
                             truncation=True,
                             return_token_type_ids=True,
                             return_attention_mask=True,
                             return_tensors = "pt")
        
        data = {'input_ids': tokenized_text['input_ids'].clone().detach().long(),
               'attention_mask': tokenized_text['attention_mask'].clone().detach().long(),
               'token_type_ids': tokenized_text['token_type_ids'].clone().detach().long(),
               }
        
        if self.mode != 'test':
            label = self.data.label.iloc[idx]
            return data, label
        else:
            return data
        

    
train_dataset = CustomDataset(train_df, TOKENIZER, args.max_seq_len, mode ='train')
print("train dataset loaded.")

In [ ]:
if DEBUG ==True :
    print("dataset sample : ")
    print(train_dataset[0])

In [ ]:
# encoded_plus = tokenizer.encode_plus(
#                     sentence,                      # Sentence to encode.
#                     add_special_tokens = True, # Add '[CLS]' and '[SEP]'
#                     max_length = 128,           # Pad & truncate all sentences.
#                     pad_to_max_length = True,
#                     return_attention_mask = True,   # Construct attention masks.
#                     return_tensors = 'pt',     # Return pytorch tensors.
#                )

### 3-2. Train, Validation set 나누기

In [ ]:
from sklearn.model_selection import train_test_split
                                                         
train_data, val_data = train_test_split(train_df, test_size=0.1, random_state=args.seed)

train_dataset = CustomDataset(train_data, TOKENIZER, args.max_seq_len, 'train')
val_dataset = CustomDataset(val_data, TOKENIZER, args.max_seq_len, 'validation')

print("Train dataset: ", len(train_dataset))
print("Validation dataset: ", len(val_dataset))

## 4. 분류 모델 학습을 위한 세팅

### 4-1. BertForSequenceClassification 설정


(https://huggingface.co/docs/transformers/v4.16.2/en/main_classes/configuration#transformers.PretrainedConfig.from_pretrained)

[PretrainedConfig](https://huggingface.co/transformers/v3.0.2/main_classes/configuration.html)


In [ ]:
from transformers import logging
logging.set_verbosity_error()

# config.json 에 입력된 architecture 에 따라 베이스 모델 설정
BASE_MODELS = {
    "BertForSequenceClassification": BertForSequenceClassification
}


myModel = BASE_MODELS[args.architecture].from_pretrained(args.pretrained_model, 
                                                         num_labels = args.num_classes, 
                                                         output_attentions = False, # Whether the model returns attentions weights.
                                                         output_hidden_states = True # Whether the model returns all hidden-states.
                                                        )
if DEBUG==True:
    # 모델 구조 확인
    print(myModel)

In [ ]:
# !pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

### 4-2. 모델 설정


BertForSequenceClassifier (line 1232부터 참고) [source code](https://github.com/huggingface/transformers/blob/a39dfe4fb122c11be98a563fb8ca43b322e01036/src/transformers/modeling_bert.py#L1284-L1287)



In [ ]:
class myClassifier(nn.Module):
    def __init__(self, model, hidden_size = 768, num_classes=args.num_classes, dr_rate=None, params=None):
        super(myClassifier, self).__init__()
        self.model = model
        self.softmax = nn.Softmax(dim=1)

    def forward(self, token_ids, attention_mask, segment_ids):      
        outputs = self.model(input_ids = token_ids, 
                             token_type_ids = segment_ids.long(), 
                             attention_mask = attention_mask.float().to(token_ids.device))
         
        logits = outputs.logits
        output = self.softmax(logits)
        return output
        
model = myClassifier(myModel, dr_rate=0.1)

# if DEBUG ==True :
#     print(model)

### 4-3. 모델 구성 확인

In [ ]:
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

## 5. 학습 진행

### 5-0. Early Stopper 함수 정의

In [ ]:
class LossEarlyStopper():
    """Early stopper

        patience (int): loss가 줄어들지 않아도 학습할 epoch 수
        patience_counter (int): loss 가 줄어들지 않을 때 마다 1씩 증가
        min_loss (float): 최소 loss
        stop (bool): True 일 때 학습 중단

    """

    def __init__(self, patience: int)-> None:
        """ 초기화

        Args:
            patience (int): loss가 줄어들지 않아도 학습할 epoch 수
            weight_path (str): weight 저장경로
            verbose (bool): 로그 출력 여부, True 일 때 로그 출력
        """
        self.patience = patience
        self.patience_counter = 0
        self.min_loss = np.Inf
        self.stop = False

    def check_early_stopping(self, loss: float)-> None:
        msg = ''
        # 첫 에폭
        if self.min_loss == np.Inf:
            self.min_loss = loss
           
        # loss가 줄지 않는다면 -> patience_counter 1 증가
        elif loss > self.min_loss:
            self.patience_counter += 1
            msg = f"Early stopping counter {self.patience_counter}/{self.patience}"

            # patience 만큼 loss가 줄지 않았다면 학습을 중단합니다.
            if self.patience_counter == self.patience:
                self.stop = True
  
        # loss가 줄어듬 -> min_loss 갱신, patience_counter 초기화
        elif loss <= self.min_loss:
            self.patience_counter = 0
            self.save_model = True
            msg = f"Validation loss decreased {self.min_loss} -> {loss}"
            self.min_loss = loss

        print(msg)

### 5-1. Epoch 별 학습 및 검증

- Adam optimizer의 epsilon 파라미터 eps = 1e-8 는 "계산 중 0으로 나눔을 방지 하기 위한 아주 작은 숫자 " 입니다. ([출처](https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/))
- `warmup_ratio` : 
  - 학습이 진행되면서 학습률을 그 상황에 맞게 가변적으로 적당하게 변경되게 하기 위해 Scheduler를 사용합니다.
  - 처음 학습률(Learning rate)를 warm up하기 위한 비율을 설정하는 warmup_ratio을 설정합니다.

In [ ]:
# args = set_config(config_path)

logging.set_verbosity_warning()

# 재현을 위해 모든 곳의 시드 고정
seed_val = args.seed
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def train(model, train_data, val_data, args, mode = 'train'):
    
    # args.run은 실험 이름 (어디까지나 팀원들간의 버전 관리 및 공유 편의를 위한 것으로, 자유롭게 수정 가능합니다.)
    print("RUN : ", args.run)
    shutil.copyfile("config.json", os.path.join(args.config_dir, f"config_{args.run}.json"))

    early_stopper = LossEarlyStopper(patience=args.patience)
    
    train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=args.train_batch_size, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=args.train_batch_size)

    DEBUG=False
    
    if DEBUG == True:
        # 데이터로더가 성공적으로 로드 되었는지 확인
        for idx, data in enumerate(train_dataloader):
            if idx==0:
                print("batch size : ", len(data[0]['input_ids']))
                print("The first batch looks like ..\n", data[0])
    
    
    criterion = nn.CrossEntropyLoss()
    
    total_steps = len(train_dataloader) * args.train_epochs

    optimizer = Adam(model.parameters(), lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=int(total_steps * args.warmup_proportion), num_training_steps=total_steps)

    
    if use_cuda:
        model = model.to(DEVICE)
        criterion = criterion.to(DEVICE)
        
    model.train()
    optimizer.zero_grad()

    tr_loss = 0.0
    val_loss = 0
    best_score = 0
      

    for epoch_num in range(args.train_epochs):

            total_acc_train = 0
            total_loss_train = 0
            
            assert mode in ['train', 'val'], 'your mode should be either \'train\' or \'val\''
            
            if mode =='train':
                for train_input, train_label in tqdm(train_dataloader):
                    
                    mask = train_input['attention_mask'].to(DEVICE)
                    input_id = train_input['input_ids'].squeeze(1).to(DEVICE)
                    segment_ids = train_input['token_type_ids'].squeeze(1).to(DEVICE)
                    train_label = train_label.long().to(DEVICE)                

                    output = model(input_id, mask, segment_ids)
                    
                    batch_loss = criterion(output.view(-1,6), train_label.view(-1))
                    total_loss_train += batch_loss.item()

                    acc = (output.argmax(dim=1) == train_label).sum().item()
                    total_acc_train += acc

                    model.zero_grad()
                    batch_loss.backward()
                    optimizer.step()
                    

            total_acc_val = 0
            total_loss_val = 0
            
            # validation을 위해 이걸 넣으면 이 evaluation 프로세스 중엔 dropout 레이어가 다르가 동작한다.
            model.eval()
            
            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    mask = val_input['attention_mask'].to(DEVICE)
                    input_id = val_input['input_ids'].squeeze(1).to(DEVICE)
                    segment_ids = val_input['token_type_ids'].squeeze(1).to(DEVICE)
                    val_label = val_label.long().to(DEVICE)

                    output = model(input_id, mask, segment_ids)

                    batch_loss = criterion(output.view(-1,6), val_label.view(-1))
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            
            train_loss = total_loss_train / len(train_data)
            train_accuracy = total_acc_train / len(train_data)
            val_loss = total_loss_val / len(val_data)
            val_accuracy = total_acc_val / len(val_data)
            
            # 한 Epoch 학습 후 학습/검증에 대해 loss와 평가지표 (여기서는 accuracy로 임의로 설정) 출력
            print(
                f'Epoch: {epoch_num + 1} \
                | Train Loss: {train_loss: .3f} \
                | Train Accuracy: {train_accuracy: .3f} \
                | Val Loss: {val_loss: .3f} \
                | Val Accuracy: {val_accuracy: .3f}')
          
            # early_stopping check
            early_stopper.check_early_stopping(loss=val_loss)

            if early_stopper.stop:
                print('Early stopped, Best score : ', best_score)
                break

            if val_accuracy > best_score:
            # 모델이 개선됨 -> 검증 점수와 weight 갱신
                best_score = val_accuracy
                
                # 학습된 모델을 저장할 디렉토리 및 모델 이름 지정
                SAVED_MODEL =  os.path.join(args.result_dir, f'best_{args.run}.pt')
            
                check_point = {
                    'model': model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'scheduler': scheduler.state_dict()
                }
                torch.save(check_point, SAVED_MODEL)  
              


            


train(model, train_dataset, val_dataset, args, mode = 'train')

## 6. Test dataset으로 추론 (Prediction)

In [ ]:
from torch.utils.data import DataLoader

# 테스트 데이터셋 불러오기
test_data = CustomDataset(test_df, tokenizer = TOKENIZER, max_len= args.max_seq_len, mode='test')

def test(model, SAVED_MODEL, test_data, args, mode = 'test'):


    test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=args.eval_batch_size)


    if use_cuda:

        model = model.to(DEVICE)
        model.load_state_dict(torch.load(SAVED_MODEL)['model'])


    model.eval()

    pred = []

    with torch.no_grad():
        for test_input in test_dataloader:

            mask = test_input['attention_mask'].to(DEVICE)
            input_id = test_input['input_ids'].squeeze(1).to(DEVICE)
            segment_ids = test_input['token_type_ids'].squeeze(1).to(DEVICE)

            output = model(input_id, mask, segment_ids)

            output = output.argmax(dim=1).cpu().tolist()

            for label in output:
                pred.append(label)
                
    return pred

SAVED_MODEL =  os.path.join(args.result_dir, f'best_{args.run}.pt')

pred = test(model, SAVED_MODEL, test_data, args)

In [ ]:
print("prediction completed for ", len(pred), "comments")


### 

In [ ]:
# 0-5 사이의 라벨 값 별로 bias, hate로 디코딩 하기 위한 딕셔너리
bias_dict = {0: 'none', 1: 'none', 2: 'others', 3:'others', 4:'gender', 5:'gender'}
hate_dict = {0: 'none', 1: 'hate', 2: 'none', 3:'hate', 4:'none', 5:'hate'}

# 인코딩 값으로 나온 타겟 변수를 디코딩
pred_bias = ['' for i in range(len(pred))]
pred_hate = ['' for i in range(len(pred))]

for idx, label in enumerate(pred):
    pred_bias[idx]=(str(bias_dict[label]))
    pred_hate[idx]=(str(hate_dict[label]))
print('decode Completed!')



In [ ]:
submit = pd.read_csv(os.path.join(args.data_dir,'sample_submission.csv'))

submit['bias'] = pred_bias
submit['hate'] = pred_hate
submit

In [ ]:
submit.to_csv(os.path.join(args.result_dir, f"submission_{args.run}.csv"), index=False)